In [1]:
!pip install Cython
!pip install torch
!pip install tensorflow
!pip install keras
!pip install keras.layers.recurrent
!pip install transformers
!pip install SentencePiece

import random
import csv
import os
import re
import string
from random import randint, choice

ERROR: Could not find a version that satisfies the requirement keras.layers.recurrent (from versions: none)
ERROR: No matching distribution found for keras.layers.recurrent
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 57.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 124.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 85.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.8 MB/s eta 0:00:00


In [2]:
!pip install happytransformer
!pip install --upgrade accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.1/519.1 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.7 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#os.chdir("/content/drive/My Drive/magistrska/" + cur_mapa)
data_folder = "/content/drive/My Drive/magistrska/prepared_data/separated_lists/test_data.csv"

In [5]:
# Read the CSV file
with open(data_folder, 'r') as input_file:
    reader = csv.reader(input_file)
    rows = list(reader)

# Count the number of rows
num_rows = len(rows)

print(f'Number of rows: {num_rows}')

Number of rows: 47491


In [6]:
with open(data_folder, 'r') as input_file:
    reader = csv.reader(input_file)
    rows = list(reader)


# Write the second rows to a CSV file
with open('eval.csv', 'w') as second_file:
    writer = csv.writer(second_file)
    # Write the header row
    writer.writerow(["input", "target"])
    writer.writerows(rows)

In [7]:
from happytransformer import HappyTextToText

# pretrained google mt5 model
#happy_tt = HappyTextToText("T5", "google/mt5-base")

#pretrained sl t5 small model
#happy_tt = HappyTextToText("T5", "t5-sl-small", load_path="/content/drive/My Drive/magistrska/pretrained_models/t5-sl-small/t5-sl-small")

#pretrained sl t5 large model
#happy_tt = HappyTextToText("T5", "cjvt/t5-sl-large", load_path="/content/drive/My Drive/magistrska/pretrained_models/t5-sl-large")



#trained google mt5 base model
#happy_tt = HappyTextToText("T5", "google/mt5-base", load_path="/content/drive/My Drive/magistrska/ucenje_feb_15_model/")

#trained sl t5 small model
happy_tt = HappyTextToText("T5", "t5-sl-small", load_path="/content/drive/My Drive/magistrska/trained_models/char_t5_aug_3/")

#trained sl t5 large model
#happy_tt = HappyTextToText("T5", "cjvt/t5-sl-large", load_path="/content/drive/My Drive/magistrska/ucenje_mar_11_slo_t5_model_large_1")

#trained sl t5 on masked words
#happy_tt = HappyTextToText("T5", "t5-sl-small", load_path="/content/drive/My Drive/magistrska/trained_models/ucenje_maj_4_masked_words_1/")




You are using the legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


In [8]:
from happytransformer import TTSettings

beam_settings =  TTSettings(num_beams=5, min_length=1, max_length=400, top_p=10)

In [9]:
def compare_words(word1, word2, matchPercentage=60):
    word1_len = len(word1)
    word2_len = len(word2)
    min_len = min(word1_len, word2_len)
    matching_chars = 0

    for i in range(min_len):
        if word1[i].casefold() == word2[i].casefold():
            matching_chars += 1

    match_percentage = (matching_chars / min_len) * 100
    if match_percentage >= matchPercentage:
        return True
    else:
        return False

In [ ]:
limit_test_cases = True

all_sentences = 0
matched_wo_sentences = 0
non_matched_wo_sentences = 0

all_words = 0
correct_words = 0
incorrect_words = 0

all_words_in_right_wo_sentences = 0
correct_words_in_right_wo_sentences = 0
incorrect_words_in_right_wo_sentences = 0

all_wrong_right_words = 0
correct_wrong_right_words = 0
incorrect_wrong_right_words = 0

TP = 0
FP = 0
FN = 0
TN = 0

with open("eval.csv", "r") as file:
    reader = csv.reader(file)
    counter = 0

    for line in reader:
      #we skip the firest line
      if counter != 0:
        result_line = happy_tt.generate_text(line[0], args=beam_settings)

        result_line_split = result_line.text.split()
        correct_line_split = line[1].split()

        #Izlocimo besedo, ki oznacuje nalogo, ki jo model mora opraviti
        incorrect_line_split = line[0].split()

        for index in range(0, len(result_line_split)):
          if index < len(result_line_split) and index < len(correct_line_split) and index < len(incorrect_line_split):
            if correct_line_split[index] != incorrect_line_split[index]:
              all_wrong_right_words += 1

              if not compare_words(result_line_split[index], correct_line_split[index], 100):
                incorrect_wrong_right_words += 1
              else:
                correct_wrong_right_words += 1

        for index in range(0, len(result_line_split)):
            all_words += 1

            # print("correct_line_split: ", correct_line_split)
            # print("incorrect_line_split: ", incorrect_line_split)
            # print("result_line_split: ", result_line_split)

            if index < len(incorrect_line_split) and index < len(correct_line_split):
              # print("index: ", index)
              # print("correct_line_split[index]: ", correct_line_split[index])
              # print("incorrect_line_split[index]: ", incorrect_line_split[index])
              # print("result_line_split[index]: ", result_line_split[index])

              if correct_line_split[index] != incorrect_line_split[index] and compare_words(result_line_split[index], correct_line_split[index], 100):
                #print("TP")
                TP += 1
                print("correct_line_split: ", correct_line_split)
                print("incorrect_line_split: ", incorrect_line_split)
                print("result_line_split: ", result_line_split)
              elif correct_line_split[index] != incorrect_line_split[index] and not compare_words(result_line_split[index], correct_line_split[index], 100):
                # print("FN")
                FN += 1
                # print("correct_line_split: ", correct_line_split)
                # print("incorrect_line_split: ", incorrect_line_split)
                # print("result_line_split: ", result_line_split)
              elif correct_line_split[index] == incorrect_line_split[index] and compare_words(result_line_split[index], correct_line_split[index], 100):
                #print("TN")
                TN += 1
              elif correct_line_split[index] == incorrect_line_split[index] and not compare_words(result_line_split[index], correct_line_split[index], 100):
                #print("FP")
                FP += 1
                # print("correct_line_split: ", correct_line_split)
                # print("incorrect_line_split: ", incorrect_line_split)
                # print("result_line_split: ", result_line_split)
              else:
                print('NAPAKA')
              # if not compare_words(result_line_split[index], correct_line_split[index], 100):
              #   #print("padel pri: ", result_line_split[index], " in ", correct_line_split[index])
              #   incorrect_words += 1
              # else:
              #   correct_words += 1


        all_same = 1

        if len(result_line_split) == len(correct_line_split):
          for index in range(0, len(result_line_split)):
            all_words_in_right_wo_sentences += 1

            if not compare_words(result_line_split[index], correct_line_split[index], 100):
              #print("padel pri: ", result_line_split[index], " in ", correct_line_split[index])
              incorrect_words_in_right_wo_sentences += 1
            else:
              correct_words_in_right_wo_sentences += 1

            if not compare_words(result_line_split[index], correct_line_split[index]):
              all_same -= 1
              #print("padel pri: ", result_line_split[index], " in ", correct_line_split[index])

              if all_same == 0:
                break
        else:
          all_same = 0

        if all_same > 0:
          matched_wo_sentences += 1
        else:
          non_matched_wo_sentences += 1

          #print("equal length, but different words")
          #print(result_line_split)
          #print(correct_line_split)

        all_sentences += 1

      counter += 1
      if limit_test_cases and counter > 100:
        break

print("all_sentences: ", all_sentences)
print("matched_wo_sentences: ", matched_wo_sentences)
print("non_matched_wo_sentences: ", non_matched_wo_sentences)

print("all_words_in_right_wo_sentences: ", all_words_in_right_wo_sentences)
print("correct_words_in_right_wo_sentences: ", correct_words_in_right_wo_sentences)
print("incorrect_words_in_right_wo_sentences: ", incorrect_words_in_right_wo_sentences)

print("all_words: ", all_words)
print("correct_words:", correct_words)
print("incorrect_words:", incorrect_words)

print("all_wrong_right_words: ", all_wrong_right_words)
print("correct_wrong_right_words: ", correct_wrong_right_words)
print("incorrect_wrong_right_words: ", incorrect_wrong_right_words)

In [1]:
print("TP:", TP)
print("FP:", FP)
print("TN:", TN)
print("FN:", FN)

NameError: ignored

In [24]:
example_1 = "na otok odšli z avtomobilom smo ."
result_1 = happy_tt.generate_text(example_1, args=beam_settings)
print(result_1.text)

Na otok smo odšli z avtomobilom .


In [17]:
example_1 = "D v n č i c n _ d r u g a _ t e k m a _ v _ l i g i _ N B A _ č a k a _ v _ s k b o t o _ # _ k o _ b o _ t e k m e c _ M i n n s o t a _ # _ "
result_1 = happy_tt.generate_text(example_1, args=beam_settings)
print(result_1.text)

D a n č i c a _ d r u g a _ t e k m a _ v _ l i g i _ N B A _ č a k a _ v _ s k r b o t o _ # _ k o _ b o _ t e k m e c _ M i n n s o t a _ # _


In [27]:
example_1 = "grem domov v mesto novo ne ."
result_1 = happy_tt.generate_text(example_1, args=beam_settings)
print(result_1.text)

Ne grem domov v Novo mesto .
